In [ ]:
# !pip install pyperclip
# !pip install selenium

In [1]:
from selenium import webdriver
import time
import pandas as pd
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pyperclip

### driver open


In [2]:
driver = webdriver.Chrome('chromedriver.exe')  # 윈도우
time.sleep(1)

C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/1923150486.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')  # 윈도우


### driver close

In [ ]:
driver.quit()

In [3]:
with open('I:\\WorkSpace\\mykey\\Naver_pw.txt') as file:
    password = file.read()
with open('I:\\WorkSpace\\mykey\\Naver_ID.txt') as file:
    ID = file.read()

### 로그인

In [5]:
# ##사용할 변수 선언 
# #네이버 로그인 주소 
url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com' 
uid = ID 
upw = password
#네이버 로그인 페이지로 이동 
driver.get(url) 
time.sleep(2) #로딩 대기 

#아이디 입력폼 
tag_id = driver.find_element_by_name('id') 
#패스워드 입력폼 
tag_pw = driver.find_element_by_name('pw')

# id 입력 
# 입력폼 클릭 -> paperclip에 선언한 uid 내용 복사 -> 붙여넣기 
tag_id.click() 
pyperclip.copy(uid) 
tag_id.send_keys(Keys.CONTROL, 'v') 
time.sleep(1) 

# pw 입력 # 입력폼 클릭 -> paperclip에 선언한 upw 내용 복사 -> 붙여넣기
tag_pw.click() 
pyperclip.copy(upw) 
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1) 

#로그인 버튼 클릭 
login_btn = driver.find_element_by_id('log.login') 
login_btn.click() 
time.sleep(2)



C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/325579290.py:11: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  tag_id = driver.find_element_by_name('id')
C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/325579290.py:13: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  tag_pw = driver.find_element_by_name('pw')
C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/325579290.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login_btn = driver.find_element_by_id('log.login')


### 전기차 검색 느영나영

In [6]:
driver.get('https://cafe.naver.com/jejutip')
tag_search = driver.find_element_by_name('query').send_keys('전기차')
driver.find_element_by_name("query").send_keys(Keys.ENTER)
time.sleep(2)
driver.switch_to.frame("cafe_main")

C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/1265406223.py:2: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  tag_search = driver.find_element_by_name('query').send_keys('전기차')
C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/1265406223.py:3: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name("query").send_keys(Keys.ENTER)


In [7]:
driver.get('https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=20692649&search.media=0&search.searchdate=all&userDisplay=15&search.option=0&search.sortBy=date&search.searchBy=0&search.query=%C0%FC%B1%E2%C2%F7&search.viewtype=title&search.page=1')
time.sleep(2)
driver.switch_to.frame('cafe_main')
driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]')[0].get_attribute("href")

C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/2477498050.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]')[0].get_attribute("href")


'https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=20692649&page=1&inCafeSearch=true&searchBy=0&query=%EC%A0%84%EA%B8%B0%EC%B0%A8&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=1422458&referrerAllArticles=true'

### URL 주소 불러오기

In [8]:
address = []
page = 0
while page > -1:
    
    if page > 101:
        break
    
    try:
        page += 1
        driver.get(f"https://cafe.naver.com/jejutip?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10456178%26search.media=0%26search.searchdate=all%26userDisplay=15%26search.option=0%26search.sortBy=date%26search.searchBy=0%26search.query=%C0%FC%B1%E2%C2%F7%26search.viewtype=title%26search.page={page}")
        driver.switch_to.frame('cafe_main')
        try: 
            if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
                break
        except:
            pass
    
        for i in range(1,16):
            address.append(driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]')[0].get_attribute("href"))
            
    except:
        pass
len(address)
# address[0:5]

C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/1610242161.py:13: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/1610242161.py:19: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  address.append(driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]')[0].get_attribute("href"))


1492

### 게시글 내용,제목,댓글 불러오기

In [ ]:
driver.get(address[1])
time.sleep(2)
driver.switch_to.frame('cafe_main')
try :
    if driver.find_element_by_class_name("title_text")==None:
        title = '제목이없습니다.'
except:
    pass
title =driver.find_element_by_class_name("title_text").text
contents =driver.find_elements_by_tag_name('p')
content = ''
for i,item in enumerate(contents):
    content += contents[i].text+'\n'
print(title)
print(content)

In [10]:
titles = []
contentss = []
for element in address:
    try:
        driver.get(element)
        time.sleep(2)
        driver.switch_to.frame('cafe_main')
        try: 
            if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
                    break
        except:
                pass
        try :
            if driver.find_element_by_class_name("title_text").text == None:
                title = '제목이없습니다.'
                pass
            else:
                title =driver.find_element_by_class_name("title_text").text
        except:
            pass
        if driver.find_elements_by_tag_name('p')[0].text == None:
            contents =driver.find_elements_by_tag_name('b')
        else:
            contents =driver.find_elements_by_tag_name('p')
        content = ''
        for j,item in enumerate(contents):
            content += contents[j].text+'\n'
        titles.append(title)
        contentss.append(content)
    except:
        pass

C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/3889474484.py:9: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/3889474484.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("title_text").text == None:
C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/3889474484.py:18: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  title =driver.find_element_by_class_name("title_text").text
C:\Users\alsdn\AppData\Local\Temp/ipykernel_4424/3889474484.py:21: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  if driver.find_elements_by_tag

In [12]:
df=pd.DataFrame({"제목":titles ,"내용":contentss,"주소":address})
df.to_csv('느영나영전기차리뷰.csv',index=False)
df.head(3)


,제목,내용,주소
0,와 렌트를 예약해놓긴했는데,\n\n기름값 보면 휘발유 디젤말고\nLPG나 전기차로 바꿔놔야될지\n진짜 너무 고...,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
1,와 렌트를 예약해놓긴했는데,\n \n혹시 어디서 하셨나요?? 다음주에 전기차 예약했는데 한번 알아보고싶어서요...,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
2,전기차 vs 휘발유 아반뗴? 뭐가 나을까요??,\n\n기름값이며 뭐며 하면 전기차가 나을 거 같은데\n만약 사고날시 전기차가 엄청...,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
